## Integer Roots and Perfect Powers

In this notebook, we cover some methods to extract integer roots - both square roots and more general $k$th roots for $k > 2$. We also describe methods to detect perfect squares and $k$th powers.

### Integer Square Roots

First we will go over the **integer square root** of a number $n \geq 0$. This is the unique integer $m$ with $m^2 \leq n < (m + 1)^2$. In calculus, we learn how to use Newton's method to find polynomial roots. We can use use this idea to compute $m = \left\lfloor \sqrt{n} \right\rfloor$.

With appropriate starting conditions, the Newtonian iteration
$$
    x_{i + 1} = x_i - \frac{f(x_i)}{f'(x_i)}
$$
will converge to a root of $f$. Using the polynomial $f(x) = x^2 - n$ yields
$$
x_{i + 1} = x_i - \frac{x_i^2 - n}{2 x_i} = \frac{1}{2} \left( x_i + \frac{n}{x_i} \right)
$$
which we may adapt as
$$
x_{i + 1} = \left\lfloor \frac{x_i + \left\lfloor \frac{n}{x_i} \right\rfloor}{2} \right\rfloor.
$$
The sequence of values $x_i$ will decrease monotonically until we obtain $x_k = \left\lfloor \sqrt{n} \right\rfloor$ for some $k$, as long as $x_0 \geq \left\lfloor \sqrt{n} \right\rfloor$. For efficiency, it is desirable to choose our initial approximation to be as close to $\sqrt{n}$ as possible. One common choice is $2^{\left\lceil \log_{2}(n) / 2 \right\rceil}$. This leads to the following algorithm:

In [30]:
def newton(n: int) -> int:
    x = 2**((n.bit_length() + 1)// 2)
    
    while True:
        y = (x + n//x) // 2
        if y < x:
            x = y
        else:
            break
    
    return x

for n in range(1, 10000):
    m = newton(n)
    assert m**2 <= n < (m + 1)**2

The sequence of terms in Newton's method $x_{i + 1} = \frac{1}{2} \left( x_i + n/x_i\right)$ is suggestive of the interval halving method of a typical binary search. We can reformulate the algorithm accordingly.

In [31]:
def binary(n: int) -> int:
    hi = 2**((n.bit_length() + 1)//2)
    lo = n // hi
    
    while lo < hi:
        hi = (hi + lo)//2
        lo = n//hi
    
    return hi

for n in range(1, 10000):
    m = binary(n)
    assert m**2 <= n < (m + 1)**2

In [68]:
t = 10**1000 + 11**1000 + 12**1000 + 13**1000

In [69]:
r = newton(t)
assert r**2 <= t <= (r + 1)**2

In [70]:
r = binary(t)
assert r**2 <= t <= (r + 1)**2

In [71]:
timeit newton(t)

180 µs ± 508 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [72]:
timeit binary(t)

178 µs ± 300 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Combining these ideas with some error checking gives a nice algorithm for computing integer square roots for $n \geq 0$.

In [41]:
def integer_sqrt(n: int) -> int:
    """
    Returns the integer square root of n >= 0 - the integer m satisfying
    m**2 <= n < (m + 1)**2.
    
    Parameters:
        n: int (n >= 0)
    
    Examples:
        >>> integer_sqrt(10)
        3
        >>> integer_sqrt(121)
        11
    """
    if n < 0:
        raise ValueError("integer_sqrt: must have n >= 0.")
    if n <= 1:
        return n
    
    hi = 2**((n.bit_length() + 1)//2)
    lo = n // hi
    
    while lo < hi:
        hi = (hi + lo)//2
        lo = n//hi
    
    return hi

In [73]:
timeit integer_sqrt(t)

178 µs ± 557 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [74]:
integer_sqrt(10)

3

In [75]:
integer_sqrt(121)

11

### Detecting perfect squares

This method may be used to check if an integer is a perfect square: simply check if the square of the integer square root is the original integer.

In [76]:
def is_square(n: int) -> bool:
    r = integer_sqrt(n)
    return r*r == n

for k in range(100):
    if is_square(k):
        print(k)

0
1
4
9
16
25
36
49
64
81


It is possible to develop more efficient methods to detect perfect integer squares. These will be covered in a later notebook.

### Integer $k$th roots

Similar to the Newtonian iteration for square roots, we can apply the idea to the polynomial $f(x) = x^k - n$ to compute integer $k$th roots. The iteration becomes $x_{i + 1} = \frac{1}{k} ((k - 1) x + n/x^{k - 1})$, and we must change the initial approximation, but the logic is largely the same as the previously described Newtonian algorithm for square roots. See Exercise 4.11 in Crandall & Pomerance for details.

In [89]:
def integer_kth_root(n: int, k: int) -> int:
    """
    Returns the integer kth root of n >= 0 - the integer m satisfying
    m**k <= n < (m + 1)**k.
    
    Parameters:
        n: int (n >= 0)
        k: int (k >= 2)
    
    Examples:
        >>> integer_kth_root(100, 3)
        3
        >>> integer_kth_root(121)
        11
    """
    if n < 0:
        raise ValueError("integer_kth_root: must have n >= 0.")
    if k < 1:
        raise ValueError("integer_kth_root: must have k >= 2.")
    if n <= 1:
        return n
    
    x = 2**((n.bit_length())//k + 1)
    
    while True:
        y = ((k - 1)*x + n//x**(k - 1))//k
        if y < x:
            x = y
        else:
            break
            
    return x

for k in range(2, 10):
    for n in range(1000):
        r = integer_kth_root(n, k)
        assert r**k <= n < (r + 1)**k

In [90]:
timeit integer_kth_root(t, 13)

146 µs ± 315 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [99]:
integer_kth_root(100, 3)

4

In [103]:
100**(1.0/3)

4.641588833612778

In [104]:
integer_kth_root(123456789, 6)

22

In [105]:
123456789**(1.0/6)

22.31443163556209

### Detecting $k$th powers

As with the square detection described earlier, we can use this to detect perfect $k$th powers.

In [92]:
def is_kth_power(n: int, k: int) -> bool:
    r = integer_kth_root(n, k)
    return r**k == n

assert is_kth_power(9, 2)
assert is_kth_power(81, 4)
assert is_kth_power(10**6, 6)

## References

[1] H. Cohen, "A Course in Computational Algebraic Number Theory",
Springer-Verlag, New York, 2000.

[2] R. Crandall, C. Pomerance, "Prime Numbers: A Computational
Perspective", Springer-Verlag, New York, 2001.